# Generacion de los datos para los modelos de Machine Learning.
 

In [7]:
import librosa
import numpy as np
import pandas as pd
import os
from IPython.display import Audio
import librosa.display
import matplotlib.pyplot as plt

## Informacion de los Audios y Herramientas 

En este proyecto se dispone de cuatro carpetas (Anger, Fear,Joy y Sadness) que contienen audios de voz de cuatro actores diferentes, dos mujeres y dos hombres. Los audios están en idioma español y se utilizará la librería librosa debido a su capacidad para extraer características de los audios.

En etapas posteriores, se detallarán las características específicas que se extraerán de los audios para su posterior análisis y aplicación de modelos de machine learning.

### Ejemplo de los audios

In [8]:
waveform1, sample_rate1 = librosa.load("Joy2m100.wav")
waveform2, sample_rate2 = librosa.load("anger1f133.wav")
waveform3, sample_rate3 = librosa.load("Fear1m147.wav")
waveform4, sample_rate4 = librosa.load("Sadness1m123.wav")

In [9]:
print("Enojo: ")
Audio(waveform2, rate=sample_rate2)

Enojo: 


In [10]:
print("Alegria: ")
Audio(waveform1, rate=sample_rate1)

Alegria: 


In [11]:
print("Miedo: ")
Audio(waveform3, rate=sample_rate3)

Miedo: 


In [12]:
print("Tristeza: ")
Audio(waveform4, rate=sample_rate4)

Tristeza: 


### Caracteristicas a extraer de los audios.
1.MFCC (Mel-Frequency Cepstral Coefficients) son coeficientes que representan la distribución del espectro de frecuencia de una señal de audio en el dominio de la frecuencia mel. Capturan información sobre los componentes espectrales y la energía en diferentes bandas de frecuencia. Estos coeficientes reflejan características relacionadas con la entonación, la pronunciación y las características vocales que pueden estar relacionadas con las emociones expresadas en la voz.

2.Mel-scaled Spectrogram (C-Mel) es una representación visual del espectro de frecuencia de una señal de audio en el dominio de la frecuencia mel. Muestra la distribución de energía en diferentes bandas de frecuencia a lo largo del tiempo. Esta representación proporciona información sobre la variación y distribución de la energía en diferentes frecuencias, lo que puede estar relacionado con aspectos vocales y expresión emocional en la voz.

3.Spectral Contrast es una característica que calcula la diferencia en amplitud entre regiones espectrales adyacentes en una señal de audio. Proporciona información sobre la variación de energía entre diferentes componentes espectrales, lo que puede ser útil para capturar cambios en la resonancia vocal y otros aspectos relacionados con las emociones expresadas en la voz.

In [17]:
def Caracteristicas(audio):
    #Esta función se encarga de extraer las características de un audio
    vector = np.array([])
    waveform, sample_rate = librosa.load(audio)
    spectrogram = np.abs(librosa.stft(waveform))
    mfcc = np.mean(librosa.feature.mfcc(y=waveform, sr=sample_rate,n_mfcc=400).T,axis=0)
    vector = np.hstack((vector, mfcc))
    cmel = np.mean(librosa.feature.melspectrogram(y=waveform, sr=sample_rate).T,axis=0)
    vector= np.hstack((vector, cmel))
    contrast = np.mean(librosa.feature.spectral_contrast(S=spectrogram, sr=sample_rate).T,axis=0)
    vector= np.hstack((vector, contrast))
    return vector

In [34]:
def Extractor(carpeta,sentimiento):
    # Esta funcion recorre todos los adios de una carpeta y los guarda en un documento. 
    text=open(sentimiento,"w")
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".wav"):
            ruta_completa = os.path.join(carpeta, archivo)
            features=list(Caracteristicas(ruta_completa))
            features+=[sentimiento]
            datos = ",".join(str(elemento) for elemento in features)
            text.write(datos+"\n")
            features.clear()
            datos=""
    text.close()
    return 0


In [35]:
#extraemos las caracteristicas de los audios en las carpetas.
Extractor("Anger","Enojo")
Extractor("Fear","Miedo")
Extractor("Joy","Alegria")
Extractor("Sadness","Tristeza")

0

In [3]:
"""
Una vez que se han obtenido los documentos de características de los audios, se desea combinar estos documentos en uno solo 
y desordenar las filas para obtener una matriz que contenga todas las emociones mezcladas.
"""
# Rutas de los archivos de texto a unir
rutas_archivos = ["Enojo", "Miedo", "Alegria","Tristeza"]

# Leer y combinar los contenidos de los archivos
contenido_combinado = ""
for ruta_archivo in rutas_archivos:
    with open(ruta_archivo, "r") as archivo:
        contenido = archivo.read()
        contenido_combinado += contenido + "\n"

# Guardar el contenido combinado en un nuevo archivo de texto
with open("Emociones_sin_Desorden", "w") as archivo_salida:
    archivo_salida.write(contenido_combinado)
df=pd.read_csv("Emociones_sin_Desorden",header=None)
df_desordenado = df.sample(frac=1).reset_index(drop=True)
df_desordenado.to_csv("Emociones_Matriz", index=False)

In [4]:
pd.read_csv("Emociones_Matriz")

,0,1,2,3,4,5,6,7,8,9,...,254,255,256,257,258,259,260,261,262,263
0,-362.673553,72.075546,-34.423786,46.613289,-31.032152,15.266807,-20.625690,2.184793,-9.916306,-3.980718,...,4.414119e-10,4.373447e-10,24.279842,16.622401,17.734212,15.988120,15.933145,17.518805,66.666761,Alegria
1,-393.728394,67.015053,-46.419445,47.776947,-30.041224,9.602686,-14.346268,1.532115,-8.717780,-3.023196,...,7.632326e-10,7.548238e-10,26.246607,15.571726,16.792642,15.534656,16.001114,19.062903,65.252453,Alegria
2,-317.546478,93.726006,-37.892796,45.128754,-30.400118,14.355480,-22.514400,-5.387063,-16.479059,-5.630636,...,3.986328e-09,3.953883e-09,23.819311,13.924518,16.490391,15.291563,15.968700,16.557738,63.421690,Alegria
3,-338.272797,101.207344,-18.354639,53.104992,-21.249622,21.387583,-11.583541,3.776414,2.967414,-8.288701,...,4.080044e-10,4.008715e-10,24.451278,12.779179,15.524099,14.246878,15.174718,17.015071,61.576324,Tristeza
4,-376.405365,76.822441,-45.807087,37.414188,-23.127131,12.868376,-22.226809,0.356228,-11.743480,-2.521581,...,7.668609e-10,7.576563e-10,25.353017,15.016093,16.757970,15.196745,16.034216,17.078185,66.036365,Alegria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,-275.033966,115.461601,-35.839348,48.204117,-20.765072,5.289915,-17.081821,-5.701774,-13.581367,-11.612162,...,2.242420e-12,1.399975e-12,22.070031,17.198062,19.947650,16.422382,16.589549,18.405972,62.048563,Miedo
2250,-372.512726,98.007347,-16.162790,49.410316,-25.252052,12.133204,-13.785635,-3.715836,-18.385136,-12.829364,...,9.588834e-10,9.457424e-10,24.974493,16.579856,20.845648,16.504716,15.294603,17.840430,64.968030,Tristeza
2251,-359.935791,98.986351,-21.551836,53.981857,-16.735882,15.440318,-15.904856,-1.836725,-9.327861,-5.195196,...,4.644489e-10,4.578064e-10,28.158091,15.522315,17.886789,14.728448,15.644271,17.016431,64.147241,Miedo
2252,-333.350861,78.212143,-55.106045,40.040394,-38.437691,8.634222,-17.049152,-14.381535,-14.443763,-6.842310,...,7.433813e-10,7.351912e-10,25.121354,18.507936,20.001745,17.741704,17.058226,19.523310,64.777812,Alegria
